<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

# HarvestStat Data Profiling - Ethiopia

In [1]:
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import retreive_fdw_data, save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import product_name_mapping, FDW_PD_CaliSeasonYear, FDW_PD_MergeCropProductionSystem
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Set loading parameters ------------------------ #
country_name = 'Ethiopia'
country_iso2 = 'ET'
use_api_data = False
requires_authentication = False
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:32637' # Ethiopia
fn_shapes = sorted(glob.glob(f"../data/shapefile/fewsnet/{country_iso2}_Admin?_????.shp"))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
# PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# Load FDW data -------------------------------- #
df = retreive_fdw_data(country_name, country_iso2, use_api_data, requires_authentication)
# ----------------------------------------------- #

Data is loaded from the local file.
../data/crop_raw_data/adm_crop_data_raw_ET.csv is loaded.


In [3]:
#df[((df.fnid=='ET1994A20609')&(pd.to_datetime(df.period_date).dt.to_period('Y')=='2000'))]
df.loc[((df.fnid=='ET1994A20724')&(df['product']=='Maize (Corn)')&
    (df['indicator']=='Area Planted')&(df['status']=='Collected')&
    (pd.to_datetime(df.period_date).dt.to_period('Y')=='2000'))]

df.loc[((df.fnid=='ET1994A20724')&(df['product']=='Maize (Corn)')&
    (df['indicator']=='Area Planted')&(df['status']=='Collected')&
    (pd.to_datetime(df.period_date).dt.to_period('Y')=='2000')),'value']

6170    44000.0
Name: value, dtype: float64

In [4]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting
# a.1) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# a.2) Early in the record the catch-all crop production system is 'all' rather than none. Make these to none-type
df.loc[df['crop_production_system'] == 'all', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #

#Drop the blank (nan) rows that are duplicates of rows that do have data
df=df[np.isfinite(df.value)]
#limit the data to admin 2 resolution, note that this drops the Belg data
df=df.dropna(subset=['admin_2'])
rows = ['fnid','season_year', 'product','indicator','crop_production_system']

#The Maize Grain (Fresh) crop code only exists in 2020, when the yield issue was fixed, but there are now duplicates of the data
# drop the Maize (Corn) data in 2020, then rename the crop data for Maize Grain (Fresh) to Maize (corn)
df = df[~((df['product'] == 'Maize (Corn)')&(pd.to_datetime(df.period_date).dt.year==2020))]
df.loc[(df['product'] == 'Maize Grain (Fresh)'),'product'] = 'Maize (Corn)'

#Barley (White) only exists in years/areas where Barley (Unspecified) does not exist. Rename to a single Barley (Unspecified) class
df.loc[(df['product'] == 'Barley (White)'),'product'] = 'Barley (Unspecified)'

#Drop 2018 until it is fixed (incorrect conversion from quintals to tons I think)
df = df[~(pd.to_datetime(df.period_date).dt.to_period('Y')=='2018')]

## Note: I suspect the following two calues are actually the same thing

#drop observations from Kemashi in 2000. The only crop with data (sorghum) is missing ha, although it exists in the source doc
df = df[~((df.fnid=='ET1994A20609')&(pd.to_datetime(df.period_date).dt.to_period('Y')=='2000'))]


#replace a mistyped value of 44000 with 440 (confirmed with sourcr docs)
df.loc[((df.fnid=='ET1994A20724')&(df['product']=='Maize (Corn)')&
    (df['indicator']=='Area Planted')&(df['status']=='Collected')&
    (pd.to_datetime(df.period_date).dt.to_period('Y')=='2000')),'value']=440


# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([ET_Admin2_1994,ET_Admin2_2001,ET_Admin2_2003,ET_Admin2_2007,ET_Admin2_2008,ET_Admin2_2014,ET_Admin2_2019,ET_Admin2_2020], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 46,627
Removed 813 "Missing Value" points
16,422/16,580 "Area Planted" points are retained.
14,801/15,044 "Quantity Produced" points are retained.
14,591/15,003 "Yield" points are retained.
Current data points: 45,814



- Minor changes are applied ------------------- #

- Basic information --------------------------- #


Data period: 1998 - 2019
58 grain types are found: Avocado (Hass), Avocado (unspecified), Banana (unspecified), Barley (Unspecified), Beans (Red), Beans (White), Beans (mixed), Beet, Cabbage (Unspecified), Carrots, Chick Peas, Chili pepper (Unspecified), Coffee (unspecified), Enset, Ethiopian cabbage, Fava bean, Fenugreek, Field Peas, Garlic, Gibto, Grass Pea, Green Peppers, Groundnuts (In Shell), Guava (unspecified), Hops, Hot red pepper, Kabuli chick pea, Lemon (unspecified), Lentils, Lettuce (Unspecified), Linseed (unspecified), Maize (Corn), Mango (unspecified), Millet (Finger), Mixed Teff, Mung bean (unspecified), Neug, Oats (Unspecified), Onions, Orange (unspecified), Papaya (unspecified), Pineapple (unspecified), Potato (unspecified), Rape, Rice (Paddy), Safflower Seed, Sesame Seed, Sorghum, Soybean (unspecified), Sugarcane (for sugar), Sunflower Seed, Sweet Potatoes, Swiss Chard, Taro/Cocoyam (Unspecified), Tomato, Vetch, Wheat Grain, Yams
1 seasons are found: Meher (10-01)
1 c

- Recorded years per season ------------------- #
Meher: 22 years (1998-2019) (missing: 3 years)

- Number of "Quantity Produced" data points --- #
ET1994A2: 1,453 data points are found.
ET2001A2: 1,075 data points are found.
ET2003A2: 3,412 data points are found.
ET2007A2: 1,168 data points are found.
ET2008A2: 4,734 data points are found.


ET2014A2: 2,033 data points are found.
ET2019A2: 926 data points are found.

- Districts with population group(s) ---------- #
0 districts includes 'population_group'.

Table of available data


----------------------------------------------- #
Crop type: Avocado (Hass)
----------------------------------------------- #
season_name             Meher                     
crop_production_system   none                     
fnid_short             2003A2 2007A2 2008A2 2014A2
year                                              
2004                    18/73    NaN    NaN    NaN
2005                    16/73    NaN    NaN    NaN
2006                    24/73    NaN    NaN    NaN
2007                      NaN  23/75    NaN    NaN
2008                      NaN    NaN  23/80    NaN
2009                      NaN    NaN  26/80    NaN
2010                      NaN    NaN    NaN    NaN
2011                      NaN    NaN  24/80    NaN
2012                      NaN    NaN  26/80    NaN
2013                      NaN    NaN  27/80    NaN
2014                      NaN    NaN    NaN    NaN
2015                      NaN    NaN    NaN  29/77
2016                      NaN    NaN    NaN  28/77
-------

season_name             Meher                                          
crop_production_system   none                                          
fnid_short             1994A2 2001A2 2003A2 2007A2 2008A2 2014A2 2019A2
year                                                                   
1998                     2/66    NaN    NaN    NaN    NaN    NaN    NaN
1999                     8/66    NaN    NaN    NaN    NaN    NaN    NaN
2000                     4/66    NaN    NaN    NaN    NaN    NaN    NaN
2001                     3/66    NaN    NaN    NaN    NaN    NaN    NaN
2002                      NaN  13/68    NaN    NaN    NaN    NaN    NaN
2003                      NaN    NaN    NaN    NaN    NaN    NaN    NaN
2004                      NaN    NaN   7/73    NaN    NaN    NaN    NaN
2005                      NaN    NaN   6/73    NaN    NaN    NaN    NaN
2006                      NaN    NaN  11/73    NaN    NaN    NaN    NaN
2007                      NaN    NaN    NaN  11/75    NaN    NaN

season_name             Meher                                   
crop_production_system   none                                   
fnid_short             2001A2 2003A2 2007A2 2008A2 2014A2 2019A2
year                                                            
2002                    36/68    NaN    NaN    NaN    NaN    NaN
2003                      NaN    NaN    NaN    NaN    NaN    NaN
2004                      NaN  32/73    NaN    NaN    NaN    NaN
2005                      NaN  29/73    NaN    NaN    NaN    NaN
2006                      NaN  38/73    NaN    NaN    NaN    NaN
2007                      NaN    NaN  33/75    NaN    NaN    NaN
2008                      NaN    NaN    NaN  31/80    NaN    NaN
2009                      NaN    NaN    NaN  34/80    NaN    NaN
2010                      NaN    NaN    NaN   2/80    NaN    NaN
2011                      NaN    NaN    NaN  35/80    NaN    NaN
2012                      NaN    NaN    NaN  33/80    NaN    NaN
2013                     

../figures/ET_admin_shapes.png is saved.


- Ethiopia crop seasonal calendar

<img src="https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-ethiopia.png?itok=xWM8Nlr6"  width=900><br>

<!-- <img src="https://www.fao.org/giews/countrybrief/country/ETH/graphics/1_2023-06-15.jpg"  width=900> -->

- comparison of boundaries

<img src="../figures/ET_admin_shapes.png"  width=900>

- In Ethiopia, basd on the FEWS NET data, there were admin unit changes in 1994, 2001, 2003, 2007, 2008, 2014, 2019, and 2020

| year	 | Admin1   | # units   | Admin2   | # units   |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1994	 | ET1994A1 | 11	| ET1994A2	| 66	| nan	| 0	|
| 2001	 | ET2001A1 | 11	| ET2001A2	| 68	| nan	| 0	|
| 2003	 | ET2003A1 | 11	| ET2003A2	| 73	| nan	| 0	|
| 2007	 | ET2007A1 | 11	| ET2007A2	| 75	| nan	| 0	|
| 2008	 | ET2008A1 | 11	| ET2008A2	| 80	| nan	| 0	|
| 2014	 | ET2014A1 | 11	| ET2014A2	| 77	| nan	| 0	|
| 2019	 | ET2019A1 | 11	| ET2019A2	| 84	| nan	| 0	|
| 2020	 | ET2020A1 | 11	| ET2020A2	| 91	| nan	| 0	|

- There is a 2020 shapefile, but no data associated with it so we use 2019 as the most recent

In [5]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = ET_Admin2_2014.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

ET1994A21706:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).
ET1994A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2001A20406:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).
ET2001A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2003A20406:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).
ET2003A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2007A20406:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).


ET2007A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2008A20406:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).
ET2008A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2014A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa" (shapefile).
ET2014A20406:	"North Shewa (OR)" (FDW) is changed to "North Shewa" (shapefile).
ET2019A21202:	"Agniwak" (FDW) is changed to "Nuer" (shapefile).
ET2019A20420:	"Finfine" (FDW) is changed to "Finfinne Special" (shapefile).
ET2019A21203:	"Majang" (FDW) is changed to "Mezhenger" (shapefile).
ET2019A21201:	"Nuer" (FDW) is changed to "Agniwak" (shapefile).
ET2019A20203:	"Gabi" (FDW) is changed to "Zone 3" (shapefile).
ET2019A20305:	"North Shewa (AM)" (FDW) is changed to "North Shewa-Am" (shapefile).


In [6]:
# Link admin boundaries ------------------------- #
link_1994, over_1994 = FDW_PD_CreateAdminLink(ET_Admin2_1994, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2001, over_2001 = FDW_PD_CreateAdminLink(ET_Admin2_2001, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2003, over_2003 = FDW_PD_CreateAdminLink(ET_Admin2_2003, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2007, over_2007 = FDW_PD_CreateAdminLink(ET_Admin2_2007, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2008, over_2008 = FDW_PD_CreateAdminLink(ET_Admin2_2008, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)

# Crop specific ratios
link_ratio_1994 = FDW_PD_RatioAdminLink(link_1994, prod, over_1994, mdx_pss)
link_ratio_2001 = FDW_PD_RatioAdminLink(link_2001, prod, over_2001, mdx_pss)
link_ratio_2003 = FDW_PD_RatioAdminLink(link_2003, prod, over_2003, mdx_pss)
link_ratio_2007 = FDW_PD_RatioAdminLink(link_2007, prod, over_2007, mdx_pss)
link_ratio_2008 = FDW_PD_RatioAdminLink(link_2008, prod, over_2008, mdx_pss)


# Merge link_ratio
assert link_ratio_1994.keys() == link_ratio_2008.keys()
assert link_ratio_2001.keys() == link_ratio_2008.keys()
assert link_ratio_2003.keys() == link_ratio_2008.keys()
assert link_ratio_2007.keys() == link_ratio_2008.keys()

link_merged = [link_ratio_1994, link_ratio_2001,link_ratio_2003,link_ratio_2007,link_ratio_2008]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #


CBR is considered for 'ET1994A21603' as no record found in: ['ET2014A20503']
CBR is considered for 'ET1994A21604' as no record found in: ['ET2014A20504']
CBR is considered for 'ET1994A21614' as no record found in: ['ET2014A20507']
CBR is considered for 'ET1994A20202' as no record found in: ['ET2014A20202']
CBR is considered for 'ET1994A20204' as no record found in: ['ET2014A20204']
CBR is considered for 'ET1994A20608' as no record found in: ['ET2014A20605']


CBR is considered for 'ET1994A20205' as no record found in: ['ET2014A20205']
CBR is considered for 'ET1994A21410' as no record found in: ['ET2014A21410']


CBR is considered for 'ET2001A20508' as no record found in: ['ET2014A20508']
CBR is considered for 'ET2001A20503' as no record found in: ['ET2014A20503']
CBR is considered for 'ET2001A20504' as no record found in: ['ET2014A20504']
CBR is considered for 'ET2001A20506' as no record found in: ['ET2014A20506']
CBR is considered for 'ET2001A20505' as no record found in: ['ET2014A20505']
CBR is considered for 'ET2001A20507' as no record found in: ['ET2014A20507']
CBR is considered for 'ET2001A20202' as no record found in: ['ET2014A20202']
CBR is considered for 'ET2001A20204' as no record found in: ['ET2014A20204']
CBR is considered for 'ET2001A20608' as no record found in: ['ET2014A20605']


CBR is considered for 'ET2001A20205' as no record found in: ['ET2014A20205']


CBR is considered for 'ET2001A21410' as no record found in: ['ET2014A21410']


CBR is considered for 'ET2003A20508' as no record found in: ['ET2014A20508']
CBR is considered for 'ET2003A20503' as no record found in: ['ET2014A20503']
CBR is considered for 'ET2003A20504' as no record found in: ['ET2014A20504']
CBR is considered for 'ET2003A20506' as no record found in: ['ET2014A20506']
CBR is considered for 'ET2003A20505' as no record found in: ['ET2014A20505']
CBR is considered for 'ET2003A20507' as no record found in: ['ET2014A20507']
CBR is considered for 'ET2003A20202' as no record found in: ['ET2014A20202']
CBR is considered for 'ET2003A20204' as no record found in: ['ET2014A20204']
CBR is considered for 'ET2003A20608' as no record found in: ['ET2014A20605']
CBR is considered for 'ET2003A20205' as no record found in: ['ET2014A20205']
CBR is considered for 'ET2003A21410' as no record found in: ['ET2014A21410']


CBR is considered for 'ET2007A20508' as no record found in: ['ET2014A20508']
CBR is considered for 'ET2007A20503' as no record found in: ['ET2014A20503']
CBR is considered for 'ET2007A20504' as no record found in: ['ET2014A20504']
CBR is considered for 'ET2007A20506' as no record found in: ['ET2014A20506']
CBR is considered for 'ET2007A20505' as no record found in: ['ET2014A20505']
CBR is considered for 'ET2007A20507' as no record found in: ['ET2014A20507']
CBR is considered for 'ET2007A20202' as no record found in: ['ET2014A20202']
CBR is considered for 'ET2007A20204' as no record found in: ['ET2014A20204']
CBR is considered for 'ET2007A20608' as no record found in: ['ET2014A20605']
CBR is considered for 'ET2007A20205' as no record found in: ['ET2014A20205']
CBR is considered for 'ET2007A21410' as no record found in: ['ET2014A21410']


CBR is considered for 'ET2008A20508' as no record found in: ['ET2014A20508']
CBR is considered for 'ET2008A20503' as no record found in: ['ET2014A20503']
CBR is considered for 'ET2008A20504' as no record found in: ['ET2014A20504']
CBR is considered for 'ET2008A20506' as no record found in: ['ET2014A20506']
CBR is considered for 'ET2008A20505' as no record found in: ['ET2014A20505']
CBR is considered for 'ET2008A20507' as no record found in: ['ET2014A20507']
CBR is considered for 'ET2008A20202' as no record found in: ['ET2014A20202']
CBR is considered for 'ET2008A20204' as no record found in: ['ET2014A20204']
CBR is considered for 'ET2008A20605' as no record found in: ['ET2014A20605']
CBR is considered for 'ET2008A20205' as no record found in: ['ET2014A20205']
CBR is considered for 'ET2008A21410' as no record found in: ['ET2014A21410']


In [7]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Product name mapping
stack = product_name_mapping(stack, list_except=None)
# Calibration of crop calendar using External Season Calendar (ESC)
esc = pd.read_csv('../data/crop_calendar/external_season_calendar.csv')
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, esc, link_ratio=None)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_ET.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_npz('../data/crop/adm_crop_production_ET_ratio.npz', link_ratio)

All [country, season_name] are in the external season calendar.


../data/crop/adm_crop_production_ET.csv is saved.
../data/crop/adm_crop_production_ET_ratio.npz is saved.


# Visualization of production data

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

def PlotBarProduction(df, year, footnote, fn_save=None):
    # Filtering and preparing data
    indicator_exist = df['indicator'].unique()
    indicator_exist = indicator_exist[~np.isin(indicator_exist, 'yield')]
    table = df.pivot_table(
        index='year',          
        columns=['fnid','country','name','product','season_name','harvest_month','indicator'],         
        values='value'
    )
    # National production
    nat = df.groupby(['season_name','product','indicator','year']).sum(min_count=1).reset_index()
    # National production in percentage
    container = []
    for (indicator, season_name) in product(indicator_exist, df.season_name.unique()):
        temp = table.loc[:, pd.IndexSlice[:,:,:,:,season_name,:,indicator]].groupby('product', axis=1).sum(min_count=1)
        temp = temp.div(temp.sum(1), axis=0) * 100
        temp = temp.stack().reset_index().rename({0: 'value'}, axis=1)
        temp['season_name'] = season_name
        temp['indicator'] = indicator
        container.append(temp)
    natp = pd.concat(container, axis=0).reset_index(drop=True)
    natp = natp[['season_name','product','indicator','year','value']]
    # Aggregation
    nat['type'] = 'orig_unit'
    natp['type'] = 'percent'
    both = pd.concat([nat, natp], axis=0)
    both = both[
        (both['indicator'] == 'production') &
        (both['season_name'] == season_name)
    ]
    # Plotting
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 6), sharex=True, gridspec_kw={'height_ratios': [1, 1]})
    fig.patch.set_facecolor('white')
    # Seasonal production (t)
    data_orig = both[both['type'] == 'orig_unit'].pivot_table(index='year', columns='product', values='value', aggfunc='sum')
    product_rank = data_orig.mean(0).sort_values(ascending=False).index
    data_orig = data_orig.reindex(np.arange(year[0], year[1] + 1)).fillna(np.nan)
    data_orig = data_orig[product_rank].sort_index()
    # Seasonal production (%)
    data_percent = both[both['type'] == 'percent'].pivot_table(index='year', columns='product', values='value', aggfunc='sum')
    data_percent = data_percent.reindex(np.arange(year[0], year[1] + 1)).fillna(np.nan)
    data_percent = data_percent[product_rank].sort_index()
    # Plotting settings
    ax = axes[0]
    data_orig.plot(ax=ax, kind='bar', stacked=True, width=0.8, colormap='tab20', alpha=1.0, legend=False, zorder=1)
    ax.set_ylabel('Production (t)')
    ax.set_ylim(0, data_orig.sum(1).max() * 1.1)
    ax.grid(which='major', axis='both', color='k', linewidth=0.3, linestyle='-', alpha=0.2, zorder=1)
    ax = axes[1]
    data_percent.plot(ax=ax, kind='bar', stacked=True, width=0.8, colormap='tab20', alpha=1.0, legend=False, zorder=1)
    ax.set_ylabel('Production (%)')
    ax.set_ylim(0, 100)
    ax.grid(which='major', axis='both', color='k', linewidth=0.3, linestyle='-', alpha=0.2, zorder=1)
    # Legend settings
    legend = axes[0].legend(bbox_to_anchor=(1.01, 1.03), loc='upper left', frameon=False, labelspacing=0.2)
    # # Clip the legend
    # for handle in legend.legendHandles:
    #     handle.set_clip_on(True)
    # Set xticks
    axes[1].set_xlabel('')
    axes[1].grid(which='minor', axis='x', linestyle='-', color='black', visible=False)
    axes[0].xaxis.set_tick_params(which='both', bottom=False)
    axes[1].xaxis.set_tick_params(which='minor', bottom=False)
    # Setting frame line width
    for spine in axes[0].spines.values(): spine.set_linewidth(0.6)
    for spine in axes[1].spines.values(): spine.set_linewidth(0.6)
    # Title and footnote
    plt.figtext(0.07, 0.015, footnote, wrap=True, horizontalalignment='left', fontsize=11)
    # Layout settings
    plt.subplots_adjust(left=0.07, right=0.85, top=0.95, bottom=0.12, hspace=0.1)
    plt.show()
    # # Save figure
    # if fn_save:
    #     fig.savefig(fn_save, bbox_inches='tight', dpi=300)
    #     print(f'{fn_save} is saved.')
    return

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
# Bar chart of national crop production
country_iso, country_name = 'ET', 'Ethiopia'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df = df[df['QC_flag'] == 0] # remove outliers
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
for season_name in ['Meher']:
    footnote = 'National crop production of %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    PlotBarProduction(sub, year, footnote, fn_save)

KeyError: 'QC_flag'

In [ ]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'ET', 'Ethiopia'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize', 'Meher'],
    ['Teff', 'Meher'],
    ['Sorghum', 'Meher'],
    ['Wheat', 'Meher'],
]
for product_name, season_name in product_season:
    footnote = 'Time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    PlotLinePAY(sub, year, footnote)